In [1812]:
!pip install sktime

In [1813]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from time import sleep
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sktime.classification.kernel_based import RocketClassifier
from sktime.datatypes import convert_to
from sktime.datatypes._panel._convert import from_nested_to_3d_numpy
from sktime.datatypes import check_is_mtype


In [1814]:
nr_of_samples_in_one_batch = 100

In [1815]:
def convert(df, list_cols=None, label_col='label'):
    if list_cols is None:
        list_cols = ['acc_x', 'acc_y', 'acc_z', 'gyr_x', 'gyr_y', 'gyr_z']

    # 1. Reset index so we have a 'instance' column
    df = df.reset_index(drop=True).rename_axis('instance').reset_index()

    # 2. Explode list columns
    df = df.explode(list_cols, ignore_index=True)

    # 3. Create a 'time' column for each exploded element
    df['time'] = df.groupby('instance').cumcount()

    # 4. Set a MultiIndex of (instance, time)
    df = df.set_index(['instance', 'time'])

    # 5. Convert all non-label columns to float
    numeric_cols = [col for col in df.columns if col != label_col]
    df[numeric_cols] = df[numeric_cols].astype(float)

    return df


In [1816]:
def read_X_samples(X, nr, samples):
    return samples.iloc[(nr*X):(nr*X+X), :]
        

In [1817]:
def addEntryToDataFrame(df, data, label):
    new_entry = {f"feature_{i}": [np.array(data.iloc[:, i].values, dtype=np.float64)]  # Wrap in list
                 for i in range(data.shape[1])}
    new_entry["label"] = [label]  # Wrap label in list
    
    # Convert to DataFrame
    df_entry = pd.DataFrame(new_entry)

    # Concatenate
    df = pd.concat([df, df_entry], ignore_index=True)
    return df


In [1818]:
def concatenate_dataFrames(dF1, dF2):
    frame = [dF1, dF2]
    return pd.concat(frame, ignore_index=True)

In [1819]:
def renameFeatures(dF):
    preferred_names = ["acc_x", "acc_y", "acc_z", "gyr_x", "gyr_y", "gyr_z", "label"]
    dF.columns = preferred_names[:len(preferred_names)]
    return dF

In [1820]:
def addWholeDatasetToDataFrame(dF, data, label):
    nr_of_batches = round(data.dropna(how='all').shape[0] / nr_of_samples_in_one_batch)
    
    for i in range(nr_of_batches):
        batch = read_X_samples(nr_of_samples_in_one_batch, i, data)
        dF = addEntryToDataFrame(dF, batch, label)
    dF = renameFeatures(dF)
    return dF

In [1821]:
def readFile_addToDataFrame(filename, dF, label_name):
    motion = pd.read_csv(filename, na_filter=False, sep=r"\s+", header=None, dtype=np.float64)
    dF = addWholeDatasetToDataFrame(dF, motion, label_name)
    dF["label"] = dF["label"].astype("category")
    return dF

In [1822]:
def shuffle_instances(df):
    # Get unique instance values and shuffle them
    shuffled_instances = shuffle(df.index.levels[0])  # Shuffle the instances
    
    # Reorder the DataFrame based on shuffled instances while keeping time order intact
    df_shuffled = pd.concat([df.loc[i] for i in shuffled_instances], keys=shuffled_instances)

    return df_shuffled

In [1823]:
dF_updown = pd.DataFrame()
dF_circles = pd.DataFrame()

In [1824]:
dF_updown = readFile_addToDataFrame('up_down2.txt', dF_updown, 'Up and down')
dF_circles = readFile_addToDataFrame('circles2.txt', dF_circles, 'Circle')

In [1825]:
dF_updown['acc_x'].dtype, dF_updown['acc_x'][0].dtype, dF_updown['label'].dtype

(dtype('O'),
 dtype('float64'),
 CategoricalDtype(categories=['Up and down'], ordered=False, categories_dtype=object))

In [1826]:
print(check_is_mtype(dF_updown, "nested_univ")) # Check if type is correct for sktime

False


In [1827]:
print(type(dF_updown.iloc[0, 0]))

<class 'numpy.ndarray'>


In [1828]:
dF_updown.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   acc_x   33 non-null     object  
 1   acc_y   33 non-null     object  
 2   acc_z   33 non-null     object  
 3   gyr_x   33 non-null     object  
 4   gyr_y   33 non-null     object  
 5   gyr_z   33 non-null     object  
 6   label   33 non-null     category
dtypes: category(1), object(6)
memory usage: 1.8+ KB


In [1829]:
dF_circles

,acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z,label
0,"[-42.97, -50.78, -50.29, -42.97, -36.13, -40.5...","[1.95, 2.44, 0.0, 10.25, 7.81, 0.49, 0.98, 6.8...","[1033.2, 1032.23, 1041.5, 1035.64, 1046.88, 10...","[-1.37, -1.34, -0.43, -2.44, -1.52, -2.35, -3....","[-0.91, -2.8, -1.25, -1.25, -2.93, -0.24, -2.1...","[0.67, 1.49, -0.98, -2.04, -1.71, 0.09, -1.16,...",Circle
1,"[-57.62, -45.41, -48.34, -56.15, -62.5, -39.06...","[-2.44, 10.74, 5.86, 0.98, -5.37, 0.49, 6.84, ...","[1041.5, 1060.55, 1041.02, 1038.57, 1042.48, 1...","[0.3, -1.34, 0.15, -0.21, -1.37, -3.57, -3.41,...","[-1.55, -2.74, -2.32, -2.87, -0.7, -1.22, -2.3...","[-0.52, 1.16, -0.46, 0.46, 1.49, -2.9, -0.15, ...",Circle
2,"[-41.99, -48.83, -35.16, -48.34, -51.27, -48.3...","[-1.46, -7.32, -6.35, -2.93, -0.49, 5.86, 2.44...","[1042.97, 1042.48, 1041.5, 1045.41, 1050.78, 1...","[0.7, -2.01, 0.52, -0.06, 1.22, -1.95, -1.19, ...","[-1.1, -1.31, -1.46, -2.47, -0.82, 0.49, -0.98...","[1.62, -0.67, 1.01, -1.16, 1.65, -0.24, -0.61,...",Circle
3,"[-54.2, -45.9, -50.29, -40.04, -34.67, -44.43,...","[3.91, -8.3, -0.49, -0.98, 8.79, 5.86, -10.74,...","[1041.99, 1045.41, 1032.23, 1035.16, 1042.48, ...","[-0.95, -0.37, -1.13, -1.8, -0.4, -0.27, -0.37...","[0.09, -1.49, -1.25, 0.52, -0.21, -3.6, 0.49, ...","[-2.44, 1.07, -2.01, -0.67, 0.27, 1.65, -1.71,...",Circle
4,"[-42.48, -54.69, -56.64, -33.2, -45.9, -41.99,...","[-0.49, -7.81, -4.39, -7.81, 3.42, 12.7, -2.93...","[1039.06, 1046.39, 1042.48, 1045.9, 1035.64, 1...","[0.43, -1.71, -1.83, -2.71, -0.46, -1.55, -2.9...","[-1.34, 0.76, -1.98, 0.03, 0.82, -0.95, -0.37,...","[1.4, 0.61, -0.18, -1.22, -0.7, -0.12, -1.46, ...",Circle
5,"[-50.78, -43.46, -40.04, -57.62, -45.41, -46.8...","[-8.79, -3.42, -5.37, -5.37, -4.88, 4.39, 2.93...","[1031.25, 1055.18, 1031.74, 1032.23, 1047.85, ...","[-1.52, -1.74, -1.13, 0.37, 0.09, 1.04, -1.89,...","[-0.09, -2.53, -2.44, 0.09, -2.9, 0.18, -0.82,...","[-1.04, 1.04, 0.61, -1.01, -0.73, -1.19, 1.89,...",Circle
6,"[-210.94, -226.56, -7.81, -177.73, -324.71, -1...","[19.04, -121.58, -104.98, 169.43, -37.6, -79.5...","[1079.1, 895.51, 1018.07, 1042.48, 1035.16, 10...","[0.4, -6.74, 5.37, 7.07, -12.74, -16.46, -7.56...","[-0.88, 4.21, 0.82, -0.21, 9.66, 16.04, 23.05,...","[34.97, 24.76, 15.64, 19.24, 24.6, -3.23, -11....",Circle
7,"[-69.34, -24.41, -23.93, -24.9, -23.93, -44.92...","[-30.27, 7.32, -2.93, -0.49, -17.58, -89.84, -...","[1029.3, 1025.39, 1028.32, 1045.9, 1035.64, 10...","[-0.7, -0.98, -3.84, -2.07, -3.35, -1.19, 0.03...","[-2.38, -1.1, -1.86, 0.15, -1.71, -0.03, -0.61...","[-1.62, -1.01, -0.98, -1.92, 1.22, 7.62, 7.96,...",Circle
8,"[-8.3, -18.55, -24.41, -15.63, -20.02, -23.44,...","[-4.39, -0.49, -5.37, -9.28, -7.81, 9.28, -0.4...","[1039.55, 1046.88, 1038.09, 1053.71, 1036.62, ...","[-1.89, 0.09, -1.4, -2.1, 1.19, -1.68, -1.01, ...","[-1.07, -1.37, -5.12, -1.55, -0.18, -0.76, -2....","[0.88, 0.3, 0.49, 0.88, -1.74, 2.13, 1.07, -3....",Circle
9,"[-17.09, -14.65, -21.0, -22.95, -17.09, -23.44...","[-0.98, -7.81, -15.63, 4.88, 0.98, 6.35, 0.49,...","[1042.48, 1040.53, 1039.06, 1035.64, 1044.92, ...","[-1.43, -1.8, -0.12, -0.03, -1.55, 0.7, -1.13,...","[-0.85, -1.62, -2.2, -0.85, -2.2, -0.91, -2.62...","[0.15, -0.21, 1.98, 2.35, 0.73, 1.4, 2.59, -0....",Circle


In [1830]:
totalDataFrame = concatenate_dataFrames(dF_circles, dF_updown)

In [1831]:
dF_circles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   acc_x   34 non-null     object  
 1   acc_y   34 non-null     object  
 2   acc_z   34 non-null     object  
 3   gyr_x   34 non-null     object  
 4   gyr_y   34 non-null     object  
 5   gyr_z   34 non-null     object  
 6   label   34 non-null     category
dtypes: category(1), object(6)
memory usage: 1.9+ KB


In [1832]:
dF_updown.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   acc_x   33 non-null     object  
 1   acc_y   33 non-null     object  
 2   acc_z   33 non-null     object  
 3   gyr_x   33 non-null     object  
 4   gyr_y   33 non-null     object  
 5   gyr_z   33 non-null     object  
 6   label   33 non-null     category
dtypes: category(1), object(6)
memory usage: 1.8+ KB


In [1833]:
totalDataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   acc_x   67 non-null     object
 1   acc_y   67 non-null     object
 2   acc_z   67 non-null     object
 3   gyr_x   67 non-null     object
 4   gyr_y   67 non-null     object
 5   gyr_z   67 non-null     object
 6   label   67 non-null     object
dtypes: object(7)
memory usage: 3.8+ KB


In [1834]:
#for row in range(len(totalDataFrame)):
 #   print(totalDataFrame.iloc[row]) 

In [1835]:
totalDataFrame

,acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z,label
0,"[-42.97, -50.78, -50.29, -42.97, -36.13, -40.5...","[1.95, 2.44, 0.0, 10.25, 7.81, 0.49, 0.98, 6.8...","[1033.2, 1032.23, 1041.5, 1035.64, 1046.88, 10...","[-1.37, -1.34, -0.43, -2.44, -1.52, -2.35, -3....","[-0.91, -2.8, -1.25, -1.25, -2.93, -0.24, -2.1...","[0.67, 1.49, -0.98, -2.04, -1.71, 0.09, -1.16,...",Circle
1,"[-57.62, -45.41, -48.34, -56.15, -62.5, -39.06...","[-2.44, 10.74, 5.86, 0.98, -5.37, 0.49, 6.84, ...","[1041.5, 1060.55, 1041.02, 1038.57, 1042.48, 1...","[0.3, -1.34, 0.15, -0.21, -1.37, -3.57, -3.41,...","[-1.55, -2.74, -2.32, -2.87, -0.7, -1.22, -2.3...","[-0.52, 1.16, -0.46, 0.46, 1.49, -2.9, -0.15, ...",Circle
2,"[-41.99, -48.83, -35.16, -48.34, -51.27, -48.3...","[-1.46, -7.32, -6.35, -2.93, -0.49, 5.86, 2.44...","[1042.97, 1042.48, 1041.5, 1045.41, 1050.78, 1...","[0.7, -2.01, 0.52, -0.06, 1.22, -1.95, -1.19, ...","[-1.1, -1.31, -1.46, -2.47, -0.82, 0.49, -0.98...","[1.62, -0.67, 1.01, -1.16, 1.65, -0.24, -0.61,...",Circle
3,"[-54.2, -45.9, -50.29, -40.04, -34.67, -44.43,...","[3.91, -8.3, -0.49, -0.98, 8.79, 5.86, -10.74,...","[1041.99, 1045.41, 1032.23, 1035.16, 1042.48, ...","[-0.95, -0.37, -1.13, -1.8, -0.4, -0.27, -0.37...","[0.09, -1.49, -1.25, 0.52, -0.21, -3.6, 0.49, ...","[-2.44, 1.07, -2.01, -0.67, 0.27, 1.65, -1.71,...",Circle
4,"[-42.48, -54.69, -56.64, -33.2, -45.9, -41.99,...","[-0.49, -7.81, -4.39, -7.81, 3.42, 12.7, -2.93...","[1039.06, 1046.39, 1042.48, 1045.9, 1035.64, 1...","[0.43, -1.71, -1.83, -2.71, -0.46, -1.55, -2.9...","[-1.34, 0.76, -1.98, 0.03, 0.82, -0.95, -0.37,...","[1.4, 0.61, -0.18, -1.22, -0.7, -0.12, -1.46, ...",Circle
...,...,...,...,...,...,...,...
62,"[-69.82, -174.8, -341.31, -406.74, -286.13, -2...","[71.78, -131.35, -372.56, -536.62, -589.84, -5...","[1024.9, 1546.39, 1828.13, 2005.86, 2131.35, 2...","[108.9, 108.11, 88.6, 68.35, 33.05, -2.01, -2....","[-67.93, -87.96, -101.89, -87.07, -53.81, -47....","[-2.93, 7.29, -13.63, -27.87, -21.16, -23.81, ...",Up and down
63,"[-122.56, -74.71, -38.57, 7.81, -0.98, -114.26...","[274.41, 166.5, 139.16, 107.42, 178.22, 345.21...","[259.77, 142.09, 30.76, 6.35, -49.8, -68.36, -...","[-65.61, -42.8, -30.52, -29.12, -16.04, 12.59,...","[30.58, 44.18, 50.64, 37.93, 4.79, -23.38, -37...","[23.02, 30.43, 36.89, 38.63, 21.52, -6.86, -17...",Up and down
64,"[-153.32, -193.36, -132.32, 10.25, -106.93, -2...","[-476.07, -490.23, -624.51, -532.71, -372.07, ...","[1956.54, 2165.53, 2293.95, 1994.63, 1751.95, ...","[10.82, -7.1, -30.21, -60.12, -97.71, -107.59,...","[6.37, 58.84, 31.65, -6.71, -15.37, -10.15, 8....","[17.68, 25.21, 24.18, 6.04, -8.11, -7.07, 8.45...",Up and down
65,"[-37.11, -13.18, -82.52, -35.16, -89.36, -174....","[330.57, 314.45, 230.47, 116.7, 53.22, -56.64,...","[8.3, 141.11, 514.16, 786.13, 1224.61, 1683.11...","[67.2, 100.18, 123.78, 126.16, 125.98, 125.61,...","[-1.13, -12.07, -28.81, -52.84, -56.01, -66.65...","[9.05, 13.38, 12.65, 0.98, -26.98, -38.72, -37...",Up and down


In [1836]:
totalDataFrame = convert(totalDataFrame)

In [1837]:
totalDataFrame

acc_x   acc_y    acc_z   gyr_x  gyr_y  gyr_z        label
instance time                                                            
0        0     -42.97    1.95  1033.20   -1.37  -0.91   0.67       Circle
         1     -50.78    2.44  1032.23   -1.34  -2.80   1.49       Circle
         2     -50.29    0.00  1041.50   -0.43  -1.25  -0.98       Circle
         3     -42.97   10.25  1035.64   -2.44  -1.25  -2.04       Circle
         4     -36.13    7.81  1046.88   -1.52  -2.93  -1.71       Circle
...               ...     ...      ...     ...    ...    ...          ...
66       95     17.09  345.21   120.12  109.05 -11.01  -0.64  Up and down
         96     -4.88  309.08   415.04  141.52 -24.48   1.37  Up and down
         97     27.34  197.27   721.68  154.60 -32.01  -0.37  Up and down
         98      7.81  142.09  1186.52  158.63 -43.96  -4.27  Up and down
         99   -106.93  -36.62  1777.34  147.44 -66.68 -19.21  Up and down

[6681 rows x 7 columns]

In [1838]:
totalDataFrame_shuffled = shuffle_instances(totalDataFrame)

In [1839]:
totalDataFrame_shuffled


acc_x   acc_y    acc_z  gyr_x  gyr_y  gyr_z   label
instance time                                                      
21       0     539.55 -229.00  1056.64  -4.36  -2.59 -23.29  Circle
         1     377.93 -528.81  1160.64  10.46   0.03 -10.88  Circle
         2     402.83 -337.89   926.27   3.69   0.64   2.50  Circle
         3     376.95 -461.91  1046.88   1.92  -1.13  10.46  Circle
         4     182.13 -413.09   987.30   1.43  -5.43  16.71  Circle
...               ...     ...      ...    ...    ...    ...     ...
9        95    -14.16    0.49  1033.69  -2.44  -2.71  -0.85  Circle
         96    -24.41   -0.49  1043.46  -1.65   0.21   0.18  Circle
         97    -18.55    0.49  1041.02   0.79  -1.16  -1.46  Circle
         98    -27.83    0.49  1039.06  -1.28  -1.01   2.38  Circle
         99    -20.02    8.79  1044.43   0.49  -0.37  -0.21  Circle

[6681 rows x 7 columns]

In [1840]:
totalDataFrame_shuffled.loc[39]

,acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z,label
time,,,,,,,
0,-8.30,6.84,1034.67,-0.67,-0.06,-0.98,Up and down
1,-18.07,16.60,1039.06,-1.52,-0.27,-1.55,Up and down
2,-6.35,9.77,1034.67,-1.07,-0.37,0.49,Up and down
3,-7.81,8.30,1040.04,-1.65,-2.56,-2.87,Up and down
4,-5.37,1.95,1046.88,0.15,0.43,-1.59,Up and down
...,...,...,...,...,...,...,...
95,10.25,-17.09,1025.88,-6.89,2.16,-5.27,Up and down
96,34.67,12.21,1030.27,-7.56,1.62,-8.72,Up and down
97,15.63,23.44,1054.69,-8.32,0.98,-4.88,Up and down


In [1841]:
y_data = totalDataFrame_shuffled['label']

In [1842]:
X_data = totalDataFrame_shuffled.drop('label', axis=1)

In [1843]:
y_data

instance  time
21        0       Circle
          1       Circle
          2       Circle
          3       Circle
          4       Circle
                   ...  
9         95      Circle
          96      Circle
          97      Circle
          98      Circle
          99      Circle
Name: label, Length: 6681, dtype: object

In [1844]:
X_data

acc_x   acc_y    acc_z  gyr_x  gyr_y  gyr_z
instance time                                              
21       0     539.55 -229.00  1056.64  -4.36  -2.59 -23.29
         1     377.93 -528.81  1160.64  10.46   0.03 -10.88
         2     402.83 -337.89   926.27   3.69   0.64   2.50
         3     376.95 -461.91  1046.88   1.92  -1.13  10.46
         4     182.13 -413.09   987.30   1.43  -5.43  16.71
...               ...     ...      ...    ...    ...    ...
9        95    -14.16    0.49  1033.69  -2.44  -2.71  -0.85
         96    -24.41   -0.49  1043.46  -1.65   0.21   0.18
         97    -18.55    0.49  1041.02   0.79  -1.16  -1.46
         98    -27.83    0.49  1039.06  -1.28  -1.01   2.38
         99    -20.02    8.79  1044.43   0.49  -0.37  -0.21

[6681 rows x 6 columns]

In [1845]:
X_data_train, X_data_test, y_data_train, y_data_test = train_test_split(
    X_data, y_data, test_size=0.2, random_state=42, shuffle=False
)

In [1846]:
X_data_train

acc_x   acc_y    acc_z   gyr_x  gyr_y  gyr_z
instance time                                               
21       0     539.55 -229.00  1056.64   -4.36  -2.59 -23.29
         1     377.93 -528.81  1160.64   10.46   0.03 -10.88
         2     402.83 -337.89   926.27    3.69   0.64   2.50
         3     376.95 -461.91  1046.88    1.92  -1.13  10.46
         4     182.13 -413.09   987.30    1.43  -5.43  16.71
...               ...     ...      ...     ...    ...    ...
64       58     87.89 -601.07  2120.61  -61.37  21.59  11.89
         59      8.30 -378.91  1751.95 -102.53 -16.49   2.65
         60    -56.64 -189.94  1450.20 -120.67 -12.50   4.70
         61   -118.16  -91.80  1229.49 -122.65   0.43   9.54
         62    -93.75  -47.85  1020.02 -112.96  11.43   9.09

[5344 rows x 6 columns]

In [1847]:
y_data_train

instance  time
21        0            Circle
          1            Circle
          2            Circle
          3            Circle
          4            Circle
                     ...     
64        58      Up and down
          59      Up and down
          60      Up and down
          61      Up and down
          62      Up and down
Name: label, Length: 5344, dtype: object

In [1848]:
X_data_train.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5344 entries, (21, 0) to (64, 62)
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   acc_x   5344 non-null   float64
 1   acc_y   5344 non-null   float64
 2   acc_z   5344 non-null   float64
 3   gyr_x   5344 non-null   float64
 4   gyr_y   5344 non-null   float64
 5   gyr_z   5344 non-null   float64
dtypes: float64(6)
memory usage: 268.5 KB


In [1849]:
check_is_mtype(X_data_train, "nested_univ")

False

In [1850]:
y_data_train.info()

<class 'pandas.core.series.Series'>
MultiIndex: 5344 entries, (21, 0) to (64, 62)
Series name: label
Non-Null Count  Dtype 
--------------  ----- 
5344 non-null   object
dtypes: object(1)
memory usage: 59.7+ KB


In [1853]:
print(len(X_data_train), len(y_data_train))

5344 5344


In [1851]:
rocket = RocketClassifier(n_features_per_kernel=6, n_jobs=-1, random_state=42)

In [1852]:
rocket.fit(X_data_train, y_data_train)

ValueError: Mismatch in number of cases. Number in X = 54 nos in y = 5344